In [15]:
from ehrhart_quasi_polynomial.ehrhart_piecewise import PiecewiseEhrhartQuasiPolynomial as PEQP
import ehrhart_quasi_polynomial.ehrhart_piecewise
from sage.matrix.constructor import Matrix

In [49]:
from inspect import getsource
print(getsource(ehrhart_quasi_polynomial.ehrhart_piecewise._generate_cone_points))

def _generate_cone_points(dimension, scaler):
    """
    Return all integral points of a simplex of dimension ``dim`` scaled
    by ``scaler``. Always returns exactly `dim + scaler choose dim` points.

    TESTS::

        sage: from ehrhart_quasi_polynomial.ehrhart_piecewise import _generate_cone_points
        sage: _generate_cone_points(2, 2)
        ((0, 0), (1, 0), (0, 1), (2, 0), (1, 1), (0, 2))
    """
    origin = free_module_element(0 for k in range(dimension))
    points = (origin, )

    basis = [free_module_element(int(k == d) for k in range(dimension))
             for d in range(dimension)]

    points += tuple(sum(combi) for index in range(1, scaler+1)
                    for combi in combinations_with_replacement(basis, index))
    return points



In [48]:
import pandas as pd
import cProfile

In [101]:
def profile(M):
    pd.options.display.float_format = '{:,.4f}'.format
    p = None
    with cProfile.Profile() as pr:
        p = PEQP(M)

    df = pd.DataFrame(pr.getstats(),
                      columns=["func", "ncalls", "ccalls", "cumtime", "tottime", "callers"])
    df["cumpc"] = df["cumtime"]/df["ncalls"]
    df["totpc"] = df["tottime"]/df["ncalls"]
    df = df[["ncalls", "tottime", "totpc", "cumtime", "cumpc", "func", "callers"]] # rearrange columns

    # remove functions with tottime < 0.001
    df = df.query("tottime > 0.0001")
    return df, p

In [98]:
A = Matrix([[-1, 0], [0, -1], [1, 1]])
B = Matrix([[-1, 0], [0, -1], [1, 2]])
C = Matrix([[-1, 0], [0, -1], [1, 1], [0, 1]])
D = Matrix([[-1, 0], [0, -1], [1, 2], [0, 1]])
E = Matrix([[-1, 0], [0, -1], [1, 3], [0, 1]])

In [102]:
%time report_p, p = profile(A)

CPU times: user 50.5 ms, sys: 718 µs, total: 51.2 ms
Wall time: 55 ms


In [95]:
report_p.sort_values("tottime", ascending=False).head()

,ncalls,tottime,totpc,cumtime,cumpc,func,callers
72,3,0.0037,0.0012,0.0037,0.0012,<method 'poll' of 'select.poll' objects>,None
111,30,0.0034,0.0001,0.0039,0.0001,<code object _convert_constraint_to_ppl at 0x7...,"[(<code object <listcomp> at 0x7f90159c7680, f..."
49,14,0.0021,0.0001,0.0021,0.0001,<built-in method posix.lstat>,None
313,3,0.0019,0.0006,0.0036,0.0012,<method 'interpolation' of 'sage.rings.polynom...,"[(<code object universe at 0x7f9084583c00, fil..."
122,8,0.0015,0.0002,0.0023,0.0003,<built-in method sage.geometry.integral_points...,[(<code object inequality_generator at 0x7f901...


In [108]:
%time report_r, r = profile(C)

CPU times: user 641 ms, sys: 191 µs, total: 642 ms
Wall time: 647 ms


In [106]:
report_r.sort_values("tottime", ascending=False)[:5]

,ncalls,tottime,totpc,cumtime,cumpc,func,callers
127,296,0.1038,0.0004,0.2180,0.0007,<built-in method sage.geometry.integral_points...,"[(<code object __getitem__ at 0x7f90159a37e0, ..."
115,1212,0.0634,0.0001,0.0708,0.0001,<code object _convert_constraint_to_ppl at 0x7...,"[(<code object <listcomp> at 0x7f90159c75d0, f..."
526,85781,0.0547,0.0000,0.1085,0.0000,"<code object copy at 0x7f9098f59d10, file ""/us...","[(<built-in method builtins.issubclass>, 85781..."
388,50,0.0360,0.0007,0.0657,0.0013,<method 'interpolation' of 'sage.rings.polynom...,"[(<code object universe at 0x7f9084583c00, fil..."
257,85693,0.0340,0.0000,0.0340,0.0000,<method '__copy__' of 'sage.modules.vector_int...,None


In [ ]:
%time report_s, s = profile(E)

In [ ]:
report_s.sort_values("tottime", ascending=False)[:5]